# Data Cleaning & Feature Engineering – Loan Approval

This notebook loads raw train/test data, applies consistent cleaning and feature engineering, aligns test columns with the train set, and saves cleaned CSVs.  
It is the single source of truth for all features used in modeling and EDA.

---

## What this notebook does

- Coerces numeric types: `age`, `income`, `emp_length`, `amounts`, `rates`, `credit_history_length`
- Drops identifier column: `id`
- Maps `cb_person_default_on_file` (Y/N) → binary `cb_person_default_on_file_bin`
- Engineers new features:
  - `dti`
  - `cred_hist_per_age`
  - `age_emp_interaction`
  - `risk_factor`
  - simple flags: high income / short employment / young borrower
- Encodes categorical variables:
  - One-hot encode `person_home_ownership` and `loan_intent` (drop-first)
  - Ordinal encode `loan_grade` → `loan_grade_encoded`
- Aligns test columns with train features (missing columns filled with 0)
- Saves cleaned datasets:
  - `data/train_cleaned.csv`
  - `data/test_cleaned.csv`

---

## Notes

- Keep this notebook aligned with **modeling** and **EDA** so that all use the same engineered features.
- If you add a new feature here, re-run both EDA and model training to stay consistent.


In [9]:
import io
from pathlib import Path

import pandas as pd
import numpy as np

# Paths
DATA_DIR = Path('..') / 'data'
TRAIN_DATA_PATH = DATA_DIR / 'train.csv'
TEST_DATA_PATH = DATA_DIR / 'test.csv'
OUTPUT_PATH = DATA_DIR / 'train_cleaned.csv'
TEST_OUTPUT_PATH = DATA_DIR / 'test_cleaned.csv'

def load_csv(path: Path, desc: str):
    if not path.exists():
        print(f"{desc} not found at {path.resolve()}")
        return None
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()
    print(f"Loaded {desc} with {df.shape[0]:,} rows and {df.shape[1]} columns from {path}")
    return df

# Load data (local only; no hardcoded Kaggle paths)
train_df = load_csv(TRAIN_DATA_PATH, 'train')
test_df = load_csv(TEST_DATA_PATH, 'test')

# Use train for profiling/cleaning
df = train_df.copy() if train_df is not None else None
if df is None:
    raise FileNotFoundError('No training data found. Place train.csv in ../data/')

# Basic info
buffer = io.StringIO()
df.info(buf=buffer, show_counts=True)
print(buffer.getvalue())

# Preview
preview_rows = min(5, len(df))
with pd.option_context('display.max_columns', None):
    display(df.head(preview_rows))

# Identify numeric and categorical columns
numeric_cols = df.select_dtypes(include='number').columns
categorical_cols = df.select_dtypes(exclude='number').columns

# Summaries
if len(numeric_cols):
    display(df[numeric_cols].describe().transpose().rename_axis('feature'))

if len(categorical_cols):
    display(
        df[categorical_cols]
        .describe(include='all')
        .transpose()
        .rename_axis('feature')
    )

# Duplicates and missingness
duplicate_count = df.duplicated().sum()
print(f"Duplicate rows: {duplicate_count:,}")

missing = df.isna().sum()
if missing.any():
    missing_report = (
        pd.DataFrame({
            'missing_count': missing,
            'missing_pct': (missing / len(df) * 100).round(2),
        })
        .query('missing_count > 0')
        .sort_values('missing_pct', ascending=False)
    )
    display(missing_report)
else:
    print('No missing values detected.')

print(f"Columns ({df.shape[1]}): {df.columns.to_list()}")

# Moderate cleaning: trim strings in object columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()

# Persist cleaned train
DATA_DIR.mkdir(parents=True, exist_ok=True)
df.to_csv(OUTPUT_PATH, index=False)
print(f"Saved cleaned train to {OUTPUT_PATH.resolve()}")

# Persist cleaned test if present
if test_df is not None:
    test_cleaned = test_df.copy()
    test_cleaned.columns = test_cleaned.columns.str.strip()
    for col in test_cleaned.select_dtypes(include='object').columns:
        test_cleaned[col] = test_cleaned[col].astype(str).str.strip()
    test_cleaned.to_csv(TEST_OUTPUT_PATH, index=False)
    print(f"Saved cleaned test to {TEST_OUTPUT_PATH.resolve()}")


Loaded train with 58,645 rows and 13 columns from ..\data\train.csv
Loaded test with 39,098 rows and 12 columns from ..\data\test.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_c

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


,count,mean,std,min,25%,50%,75%,max
feature,,,,,,,,
id,58645.0,29322.000000,16929.497605,0.00,14661.00,29322.00,43983.00,58644.00
person_age,58645.0,27.550857,6.033216,20.00,23.00,26.00,30.00,123.00
person_income,58645.0,64046.172871,37931.106978,4200.00,42000.00,58000.00,75600.00,1900000.00
person_emp_length,58645.0,4.701015,3.959784,0.00,2.00,4.00,7.00,123.00
loan_amnt,58645.0,9217.556518,5563.807384,500.00,5000.00,8000.00,12000.00,35000.00
loan_int_rate,58645.0,10.677874,3.034697,5.42,7.88,10.75,12.99,23.22
loan_percent_income,58645.0,0.159238,0.091692,0.00,0.09,0.14,0.21,0.83
cb_person_cred_hist_length,58645.0,5.813556,4.029196,2.00,3.00,4.00,8.00,30.00
loan_status,58645.0,0.142382,0.349445,0.00,0.00,0.00,0.00,1.00


,count,unique,top,freq
feature,,,,
person_home_ownership,58645,4,RENT,30594
loan_intent,58645,6,EDUCATION,12271
loan_grade,58645,7,A,20984
cb_person_default_on_file,58645,2,N,49943


Duplicate rows: 0
No missing values detected.
Columns (13): ['id', 'person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'loan_status']
Saved cleaned train to C:\Users\hasti_aksoy\OneDrive\Desktop\Loan Approval Prediction\data\train_cleaned.csv
Saved cleaned test to C:\Users\hasti_aksoy\OneDrive\Desktop\Loan Approval Prediction\data\test_cleaned.csv


In [10]:
import pandas as pd


if 'id' in df.columns:
    df = df.drop(columns=['id'])


if 'cb_person_default_on_file' in df.columns:
    df['cb_person_default_on_file_bin'] = df['cb_person_default_on_file'].map({'Y': 1, 'N': 0, 'y': 1, 'n': 0}).astype('Int64')

# 3) Feature Engineering

# Debt-to-Income Ratio 
if {'loan_amnt','person_income'}.issubset(df.columns):
    df['dti'] = (df['loan_amnt'] / df['person_income'].replace(0, np.nan)).astype(float)


# Credit History per Age 
if {'cb_person_cred_hist_length','person_age'}.issubset(df.columns):
    df['cred_hist_per_age'] = (df['cb_person_cred_hist_length'] / df['person_age'].replace(0, np.nan)).astype(float)

# Interaction: Age × Employment Length
if {'person_age','person_emp_length'}.issubset(df.columns):
    df['age_emp_interaction'] = (df['person_age'] * df['person_emp_length']).astype(float)

# Interaction: Loan Amount × Interest Rate
if {'loan_amnt','loan_int_rate'}.issubset(df.columns):
    df['risk_factor'] = (df['loan_amnt'] * df['loan_int_rate']).astype(float)

# Flags
if 'person_income' in df.columns:
    df['high_income'] = (df['person_income'] > df['person_income'].median()).astype(int)

if 'person_emp_length' in df.columns:
    df['short_emp'] = (df['person_emp_length'] < 2).astype(int)

if 'person_age' in df.columns:
    df['young_borrower'] = (df['person_age'] < 25).astype(int)

# 4) Encoding categorical columns (One-Hot)
cat_cols = ['person_home_ownership','loan_intent']
for c in cat_cols:
    if c in df.columns and df[c].dtype == 'O':
        df = pd.get_dummies(df, columns=[c], prefix=c, drop_first=True)

# Loan grade: ordinal encoding (A-G)
if 'loan_grade' in df.columns:
    grade_map = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7}
    df['loan_grade_encoded'] = df['loan_grade'].map(grade_map).astype('Int64')
    df = df.drop(columns=['loan_grade'])

print("Shape after feature engineering:", df.shape)
df.head()


Shape after feature engineering: (58645, 26)


,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,cb_person_default_on_file_bin,...,young_borrower,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_encoded
0,37,35000,0.0,6000,11.49,0.17,N,14,0,0,...,0,False,False,True,True,False,False,False,False,2
1,22,56000,6.0,4000,13.35,0.07,N,2,0,0,...,1,False,True,False,False,False,True,False,False,3
2,29,28800,8.0,6000,8.90,0.21,N,10,0,0,...,0,False,True,False,False,False,False,True,False,1
3,30,70000,14.0,12000,11.11,0.17,N,5,0,0,...,0,False,False,True,False,False,False,False,True,2
4,22,60000,2.0,6000,6.92,0.10,N,3,0,0,...,1,False,False,True,False,False,True,False,False,1


In [11]:
# artifacts learned from train
income_median_train = df['person_income'].median() if 'person_income' in df.columns else None
train_cols = df.columns.tolist()
feature_cols = [c for c in train_cols if c != 'loan_status']

if 'test_df' in globals() and test_df is not None:
    # numeric coercion
    for _col in ['person_age','person_income','person_emp_length','loan_amnt','loan_int_rate','loan_percent_income','cb_person_cred_hist_length']:
        if _col in test_df.columns:
            test_df[_col] = pd.to_numeric(test_df[_col], errors='coerce')

    # drop id
    if 'id' in test_df.columns:
        test_df = test_df.drop(columns=['id'])

    # Y/N -> 0/1
    if 'cb_person_default_on_file' in test_df.columns and 'cb_person_default_on_file_bin' not in test_df.columns:
        test_df['cb_person_default_on_file_bin'] = (
            test_df['cb_person_default_on_file']
            .map({'Y':1,'N':0,'y':1,'n':0, True:1, False:0})
            .astype('Int64')
        )

    # engineered features
    if {'loan_amnt','person_income'}.issubset(test_df.columns):
        test_df['dti'] = (test_df['loan_amnt'] / test_df['person_income'].replace(0, np.nan)).astype(float)

    if {'cb_person_cred_hist_length','person_age'}.issubset(test_df.columns):
        test_df['cred_hist_per_age'] = (test_df['cb_person_cred_hist_length'] / test_df['person_age'].replace(0, np.nan)).astype(float)

    if {'person_age','person_emp_length'}.issubset(test_df.columns):
        test_df['age_emp_interaction'] = (test_df['person_age'] * test_df['person_emp_length']).astype(float)

    if {'loan_amnt','loan_int_rate'}.issubset(test_df.columns):
        test_df['risk_factor'] = (test_df['loan_amnt'] * test_df['loan_int_rate']).astype(float)

    # flags (use train-derived threshold!)
    if income_median_train is not None and 'person_income' in test_df.columns:
        test_df['high_income'] = (test_df['person_income'] > income_median_train).astype(int)

    if 'person_emp_length' in test_df.columns:
        test_df['short_emp'] = (test_df['person_emp_length'] < 2).astype(int)

    if 'person_age' in test_df.columns:
        test_df['young_borrower'] = (test_df['person_age'] < 25).astype(int)

    # One-Hot (same as before; alignment will fix column diffs)
    for c in ['person_home_ownership','loan_intent']:
        if c in test_df.columns and test_df[c].dtype == 'O':
            test_df = pd.get_dummies(test_df, columns=[c], prefix=c, drop_first=True)

    # ordinal encode
    if 'loan_grade' in test_df.columns and test_df['loan_grade'].dtype == 'O':
        grade_map = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7}
        test_df['loan_grade_encoded'] = test_df['loan_grade'].map(grade_map).astype('Int64')
        test_df = test_df.drop(columns=['loan_grade'])

    # FINAL ALIGN (simple & robust)
    ordered = feature_cols + (['loan_status'] if 'loan_status' in test_df.columns else [])
    test_df = test_df.reindex(columns=ordered, fill_value=0)

    print('Test aligned to feature columns:', len(feature_cols), 'features')


Test aligned to feature columns: 25 features


In [12]:
# Persist cleaned datasets
from pathlib import Path

DATA_DIR = Path('..') / 'data'
TRAIN_OUT = DATA_DIR / 'train_cleaned.csv'
TEST_OUT = DATA_DIR / 'test_cleaned.csv'

DATA_DIR.mkdir(parents=True, exist_ok=True)
df.to_csv(TRAIN_OUT, index=False)
print(f"Saved cleaned train to {TRAIN_OUT.resolve()}")

if 'test_df' in globals() and test_df is not None:
    test_df.to_csv(TEST_OUT, index=False)
    print(f"Saved cleaned test to {TEST_OUT.resolve()}")


Saved cleaned train to C:\Users\hasti_aksoy\OneDrive\Desktop\Loan Approval Prediction\data\train_cleaned.csv
Saved cleaned test to C:\Users\hasti_aksoy\OneDrive\Desktop\Loan Approval Prediction\data\test_cleaned.csv
